In [ ]:
import pandas as pd
import numpy as np
import joblib
import pickle
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


##### Caminho dos arquivos CSV

In [ ]:
user_csv_path = "/usr/local/airflow/artifacts/parquets"
user_target_csv_file = "users_merged_data.parquet"

articles_csv_path = "/usr/local/airflow/artifacts/parquets"
articles_target_csv_file = "articles_merged_data.parquet"

artifacts_models_path = "/usr/local/airflow/artifacts/models"
validacao_file = "/usr/local/airflow/data/validacao.csv"

##### Carregando os dados dos usuários, artigos e validacao

In [ ]:
# df_users = pd.read_csv(f"{user_csv_path}/{user_target_csv_file}")
# df_articles = pd.read_csv(f"{articles_csv_path}/{articles_target_csv_file}")
df_users = pd.read_parquet(f"{user_csv_path}/{user_target_csv_file}")
df_articles = pd.read_parquet(f"{articles_csv_path}/{articles_target_csv_file}")
df_validacao = pd.read_csv(validacao_file)


##### Preenchendo valores ausentes de 'history' com lista vazia

In [ ]:
df_users.fillna({"history": "[]"}, inplace=True)  # Se history estiver vazio, substitui por lista vazia
df_articles.fillna("", inplace=True)


In [ ]:
df_articles["issued"] = pd.to_datetime(df_articles["issued"]).astype(int) // 10**9
df_articles["modified"] = pd.to_datetime(df_articles["modified"]).astype(int) // 10**9


In [ ]:
def convert_to_mean(value):
    if isinstance(value, str):
        values = [float(x.strip()) for x in value.split(",") if x.strip().replace('.', '', 1).isdigit()]
        return np.mean(values) if values else 0
    return value

cols_to_convert = ["numberOfClicksHistory", "timeOnPageHistory", "scrollPercentageHistory", "pageVisitsCountHistory"]
for col in cols_to_convert:
    df_users[col] = df_users[col].apply(convert_to_mean)
    
df_users.head(3)

In [ ]:
scaler = MinMaxScaler()
df_users[[
    "historySize",
    "numberOfClicksHistory",
    "timeOnPageHistory",
    "scrollPercentageHistory",
    "pageVisitsCountHistory"
]] = \
    scaler.fit_transform(
        df_users[
            [
                "historySize",
                "numberOfClicksHistory",
                "timeOnPageHistory",
                "scrollPercentageHistory",
                "pageVisitsCountHistory"
            ]
        ]
    )
df_users.head(3)

In [ ]:
df_users["history"] = df_users["history"].apply(lambda x: x.split(",") if isinstance(x, str) else [])
df_users.shape

In [ ]:
encoder = LabelEncoder()
df_users["userType"] = encoder.fit_transform(df_users["userType"])


In [ ]:
# df_users["userHash"] = df_users["userId"]
# df_users["userId"] = df_users["userId"].astype("category").cat.codes
df_articles["page"] = df_articles["page"].astype("category").cat.codes

##### Garantir que todos os usuários da validação estejam incluídos


In [ ]:
# users_validacao_ids = df_validacao["userId"].unique()
# df_users_validacao = df_users[df_users["userId"].isin(users_validacao_ids)]
# df_users

In [ ]:
df_users_sampled = df_users.sample(n=6000, random_state=42)  # Amostra de 1000 usuários
df_articles_sampled = df_articles.sample(n=1000, random_state=42)  # Amostra de 1000 artigos

In [ ]:
interaction_data_sampled = []
for _, row in df_users_sampled.iterrows():
    for article in row['history']:
        interaction_data_sampled.append((row['userId'], article))

df_interactions = pd.DataFrame(interaction_data_sampled, columns=["userId", "page"])


In [ ]:
# df_interactions_sampled = df_interactions[
#     (df_interactions["userId"].isin(df_users_sampled["userId"])) &
#     (df_interactions["page"].isin(df_articles_sampled["page"]))
# ]

# Divisão em treino e validação
# df_train, df_validacaoTemp = train_test_split(df_interactions_sampled, test_size=0.2, random_state=42)

# # Garantir que o conjunto de validação contenha apenas usuários que estão no treino
# df_validacaoTemp = df_validacaoTemp[df_validacaoTemp["userId"].isin(df_train["userId"])]

# # Exibir tamanhos dos conjuntos
# print(f"Tamanho do conjunto de treino: {df_train.shape[0]}")
# print(f"Tamanho do conjunto de validação: {df_validacaoTemp.shape[0]}")

In [ ]:
df_users_sampled.to_parquet("/usr/local/airflow/artifacts/parquets/usuarios_preprocessados.parquet", index=False)
df_articles_sampled.to_parquet("/usr/local/airflow/artifacts/parquets/artigos_preprocessados.parquet", index=False)
df_interactions.to_parquet("/usr/local/airflow/artifacts/parquets/interactions.parquet", index=False)

print("✅ Pré-processamento concluído e salvo em Parquet!")

##### Collaborative Filtering

In [ ]:
interaction_matrix = df_interactions.pivot_table(index='userId', columns='page', aggfunc='size', fill_value=0)



In [ ]:
cos_sim = cosine_similarity(interaction_matrix)
cos_sim_df = pd.DataFrame(cos_sim, index=interaction_matrix.index, columns=interaction_matrix.index)

In [ ]:
print("Matriz de similaridade entre usuários:")
print(cos_sim_df.head())

##### K Means

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df_users_sampled['cluster'] = kmeans.fit_predict(df_users_sampled[['numberOfClicksHistory', 'timeOnPageHistory']])

# Exibindo clusters dos usuários
print("Clusters dos usuários:")
print(df_users_sampled[['userId', 'cluster']].head())

##### Função para calcular a precisão das recomendações

In [ ]:
def evaluate_recommendations(recommended_articles, history):
    # Verifica se history é um array NumPy e converte para lista
    if isinstance(history, np.ndarray):
        history = history.tolist()  # Converte array NumPy para lista normal
    
    # Verifica se history é uma string concatenada em vez de uma lista real
    if isinstance(history, str):
        history = history.strip("[]").replace("'", "").split()
    
    relevant_articles = [article.strip() for article in history]
    correct_recommendations = len(set(recommended_articles).intersection(set(relevant_articles)))
    total_recommendations = len(recommended_articles)
    return correct_recommendations, total_recommendations

In [ ]:
df_users_sampled.shape

In [ ]:
print(cos_sim_df.head())


In [ ]:
# teste = users.loc['956bbe18790201499004c41bf9df52c583a9020fa21cbd2e4b117f9391b130f7']
# teste
# df_validacao.shape
users_in_sim_matrix = df_validacao['userId'].isin(cos_sim_df.index).sum()
users_in_sim_matrix

##### Recomendações para usuários de validação

In [ ]:
correct_recommendations = 0
total_recommendations = 0

for _, row in df_validacao.iterrows():
    user_id = row['userId']
    # print(user_id)
    # Verificar se o user_id está presente no cos_sim_df (índice)
    if user_id not in cos_sim_df.index:
        # print(f"❌ O userId {user_id} não está presente na matriz de similaridade.")
        continue  # Pula o usuário se não estiver no cos_sim_df

    # Usuários similares
    user_similarity = cos_sim_df.loc[user_id]
    # print(user_similarity)
    similar_users = user_similarity.sort_values(ascending=False).index[1:4]

    # Artigos recomendados
    recommended_articles = []
    for similar_user in similar_users:
        similar_user_articles = df_interactions[df_interactions['userId'] == similar_user]['page']
        recommended_articles.extend(similar_user_articles)

    # Removendo duplicatas
    recommended_articles = list(set(recommended_articles))

    # Avaliar precisão das recomendações
    correct, total = evaluate_recommendations(recommended_articles, row['history'])
    correct_recommendations += correct
    total_recommendations += total

##### Calculando a precisão das recomendações

In [ ]:
print(f"correct_recommendations: {correct_recommendations}")
print(f"total_recommendations: {total_recommendations}")
precision = correct_recommendations / total_recommendations if total_recommendations > 0 else 0
print(f"✅ Precisão das recomendações: {precision:.6f}")

In [ ]:
# Recomendações para um usuário logado
user_id = df_users_sampled['userId'].iloc[0]  # Exemplo de usuário logado (pode ser qualquer userId)
user_similarity = cos_sim_df[user_id]
similar_users = user_similarity.sort_values(ascending=False).index[1:4]
print(f"Usuários mais similares ao usuário {user_id}:")
print(similar_users)

##### Sugestões de artigos baseadas em similaridade (para o usuário logado)


In [ ]:
recommended_articles = []
for similar_user in similar_users:
    similar_user_articles = df_interactions[df_interactions['userId'] == similar_user]['page']
    recommended_articles.extend(similar_user_articles)

In [ ]:
# Exibindo os artigos recomendados
recommended_articles = list(set(recommended_articles))  # Removendo duplicatas
recommended_articles = [article.strip() for article in recommended_articles]  # Removendo espaços extras
print("Artigos recomendados:")
print(recommended_articles)

In [ ]:
# Recomendações para usuários não logados: artigos populares
popular_articles = df_interactions.groupby('page').size().reset_index(name='popularity')
popular_articles = popular_articles.sort_values('popularity', ascending=False)
print("Artigos mais populares:")
print(popular_articles.head(3))

In [ ]:
joblib.dump(kmeans, f"{artifacts_models_path}/kmeans_model.pkl")

# Save cosine similarity matrix
with open(f"{artifacts_models_path}/cos_sim_matrix.pkl", "wb") as f:
    pickle.dump(cos_sim_df, f)

# Save user and article DataFrames for later use
df_users_sampled.to_pickle(f"{artifacts_models_path}/usuarios_preprocessados.pkl")
df_articles_sampled.to_pickle(f"{artifacts_models_path}/artigos_preprocessados.pkl")